In [2]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
#from dash import dcc
import dash_html_components as html
#from dash import html
from dash.dependencies import Output, Input

import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import dtale

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import graphs.plots as plots

<ipython-input-2-4b0abde45268>:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-4b0abde45268>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
# Tutorials

#https://plotly.com/python/
#https://plotly.com/python/bar-charts/#bar-charts-in-dash

In [3]:
df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
df
#fig = px.sunburst(df, path=['Renewal','EboSchema'], values='Ultimate-Prämie')
#fig.show()

,Renewal,mng,Description,Gesbezkurz,EboSchema,brchnr,brch_text_kurz,UW-Prämie,UW-Schaden,UW-Kosten,...,Abrech-Schaden,Abrech-Kosten,Abrech-Ergebnis,Ultimate-Prämie,Ultimate-Schaden,Ultimate-Kosten,Ultimate-Ergebnis,ID,Doppelte Ids,Unnamed: 21
0,2006,413,Netherlands,NN - Nationale Nederlanden,NP,4110,Fire & AP,9.800297e+04,5.901757e+04,0.000000e+00,...,0.0,0.00,92081.00,86709.327500,0.00000,0.000000,86709.327500,X00028 2006,0,NaN
1,2006,413,Netherlands,Achmea Reinsurance Company NV,NP,3100,MTP Liab.,2.661400e+04,2.377868e+04,0.000000e+00,...,0.0,0.00,25432.62,25432.500000,22722.87720,0.000000,2709.622800,X00031 2006,0,NaN
2,2006,631,Canada,Saskatchewan Government Insurance,NP,4260,Windstorm,1.585164e+05,1.063455e+05,1.521367e+04,...,0.0,11176.51,100588.51,111765.011167,0.00000,11176.501117,100588.510051,X00044 2006,0,NaN
3,2006,720,Switzerland,Interkantonaler Rückversicherungsverband,NP,4110,Fire & AP,1.227853e+05,9.415206e+04,1.227853e+04,...,0.0,12005.68,108051.14,118126.747349,0.00000,11812.674735,106314.072615,X00086 2006,0,NaN
4,2006,520,France,Caisse Meusienne d'Assurances Mutuelles,NP,4110,Fire & AP,3.303229e+04,2.337055e+04,3.248600e+03,...,42041.0,2775.50,-17059.52,32486.400000,57922.80192,3248.640000,-28685.041920,X00092 2006,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51083,2021,540,Spain,Atradius Crédito y Caución S.A. de Seguros y R...,Prop,7100,Trade Credit ST,2.250000e+05,1.169842e+05,7.425000e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X88623 2021 01,0,NaN
51084,2021,540,Spain,Atradius Crédito y Caución S.A. de Seguros y R...,Prop,7100,Trade Credit ST,1.573980e+07,7.866437e+06,6.886162e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X88679 2021 01,0,NaN
51085,2021,540,Spain,Atradius Crédito y Caución S.A. de Seguros y R...,Prop,7290,Other Surety,1.221000e+06,5.286930e+05,5.677650e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X88679 2021 02,0,NaN
51086,2021,540,Spain,Atradius Crédito y Caución S.A. de Seguros y R...,Prop,7100,Trade Credit ST,1.800000e+05,8.023680e+04,8.820000e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X88679 2021 03,0,NaN


In [6]:
fig = px.sunburst(df, path=[
    'EboSchema',
    #'brch_text_kurz',
    'Description',
    'Renewal',
    #'mng'
], values='Ultimate-Prämie')
fig.show()

# Line


Legende:
https://plotly.com/python/legend/

In [1]:
######## Parameter
loc = 'Canada'
var = 'Description'
type_g = 'Renewal'
############

df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
df_loc = df[df[var]==loc]

df_loc=df_loc.groupby(type_g).sum()
df_loc=df_loc[:len(df_loc.index.values)-2]

measure=[]
for i in range(len(df_loc.index.values)-1):
    measure.append('relativ')
measure.append('total')
###################################
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.01,
)
fig.add_trace(
     go.Waterfall(
    name = "20", orientation = "v",
    measure = measure,
    x = df_loc.index.values,
    textposition = "outside",
    #text = ["+60", "+80", "-40", "-20", "Total"],
    y = df_loc['Abrech-Ergebnis'],
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
    ),
    row=2, col=1
)
##################################
df_loc = df[df[var]==loc]

df_heat = df_loc.groupby(type_g).sum()#[['UW-Prämie','Abrech-Prämie','Ultimate-Prämie']]
df_heat = df_heat[:len(df_heat.index.values)-2]
fig.add_trace(
    go.Bar(x=df_heat.index.values, y=df_heat['UW-Prämie'],
        name="UW-Prämie", marker = {'color' : 'darkblue'}
    ),
    row=1, col=1
)
###################
fig.update_layout(
    height=300,
    width=600,
    showlegend=False,
    title_text="Prämienverlauf - "+loc,
    plot_bgcolor = "white"
)
fig.show()

NameError: name 'pd' is not defined

In [29]:
######## Parameter
loc = 'Canada'
var = 'Description'
type_g = 'Renewal'
############

df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
df_loc = df[df[var]==loc]

df_loc=df_loc.groupby(type_g).sum()
df_loc=df_loc[:len(df_loc.index.values)-2]

measure=[]
for i in range(len(df_loc.index.values)-1):
    measure.append('relativ')
measure.append('total')
###################################
fig = make_subplots(
    rows=1, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.01,
)
fig.add_trace(
     go.Waterfall(
    name = "20", orientation = "v",
    measure = measure,
    x = df_loc.index.values,
    textposition = "outside",
    #text = ["+60", "+80", "-40", "-20", "Total"],
    y = df_loc['Abrech-Ergebnis'],
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
    ),
    row=1, col=1
)

fig.update_layout(
    height=350,
    width=600,
    showlegend=False,
    title_text="Abwicklungsergebnis - "+loc,
    plot_bgcolor = "white"
)
fig

In [ ]:

#from dash import Dash, dcc, html
import os
import pandas as pd
pd.options.display.max_columns = None
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly
import jupyter_dash as jd
import dash
import graphs.plots as plots
from plotly.subplots import make_subplots

import ssl

ssl._create_default_https_context = ssl._create_unverified_context


#app = JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
#app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.COSMO])

df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')

def data_prep(df):
    type_g = 'Renewal'
    df_heat = df.groupby(type_g).sum()#[['UW-Prämie','Abrech-Prämie','Ultimate-Prämie']]
    df_heat = df_heat[:len(df_heat.index.values)-2]
    df_heat['UW-SQ']=df_heat[['UW-Prämie','UW-Schaden']].apply(lambda x: x[0]/x[1], axis=1)
    df_heat['Abrech-SQ']=df_heat[['Abrech-Prämie','Abrech-Schaden']].apply(lambda x: x[0]/x[1], axis=1)
    df_heat['Ultimate-SQ']=df_heat[['Ultimate-Prämie','Ultimate-Schaden']].apply(lambda x: x[0]/x[1], axis=1)
    return df_heat

#dfo = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

######################
app.layout = html.Div([
    html.H1('Rentabilitätsanalysis'),
    ######            
    dbc.Row([
#        dbc.Col(lg=1),
        dbc.Col([
            dbc.Label('Basis Features'),
            dcc.Dropdown(id='gini_year_dropdown',
                         placeholder='Select a year',
                         options=[{'label': year, 'value': year}
                                  for year in range(1974, 2019)]),
            dcc.Dropdown(id='gini_country_dropdown',
                         placeholder='Select one or more countries',
                         multi=True,
                         options=[{'label': country, 'value': country}
                                  for country in df['Description'].unique()]),
            dcc.Dropdown(df['Description'].unique(),
                 'Canada', #multi=True,
                 id='country',),
            dcc.Dropdown(df.EboSchema.unique(), placeholder="Select a feature",
                     multi=True, 
                     id='feature'), 
#            style={"border":"1px black solid"},
            ]),# md=12, lg=5),
        
        dbc.Col([dcc.Graph(id='graph-p')]),# md=12, lg=5),
        dbc.Col([
            dcc.Dropdown(df.brch_text_kurz.unique(), 
                 "Windstorm", #multi=True, 
                 id='segment',),
            dcc.Graph(id='graph-all')
        ]),# md=12, lg=5),
    ],align='center'),
    dbc.Row([
        dbc.Col([ 
            dbc.Label('Advances Features'),
            dcc.Dropdown(id='gini_year_dropdown-2', placeholder='Select a year',
                         options=[{'label': year, 'value': year} for year in range(1974, 2019)]),
            dcc.Dropdown(id='gini_year_dropdown-3', placeholder='Select a year',
                         options=[{'label': year, 'value': year} for year in range(1974, 2019)]),
            dcc.Dropdown(id='gini_year_dropdown-4', placeholder='Select a year',
                         options=[{'label': year, 'value': year} for year in range(1974, 2019)]),
            dcc.Dropdown(id='gini_year_dropdown-5', placeholder='Select a year',
                         options=[{'label': year, 'value': year} for year in range(1974, 2019)]),
            ]),
        dbc.Col([ dcc.Graph(id='graph-ws')]),# md=12, lg=5),
        dbc.Col([ dcc.Graph(id='graph-loss-p')]),# md=12, lg=5),        
    ],align='center'),
])

@app.callback(
    dash.dependencies.Output('graph-all', 'figure'),
    [dash.dependencies.Input('segment', 'value')])
def update_figure(segment):
    ######## Parameter
    loc = segment
    var = 'brch_text_kurz'
    type_g = 'Renewal'
    ############

    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_loc = df[df[var]==loc]

    df_loc=df_loc.groupby(type_g).sum()
    df_loc=df_loc[:len(df_loc.index.values)-2]

    measure=[]
    for i in range(len(df_loc.index.values)-1):
        measure.append('relativ')
    measure.append('total')
    ###################################
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.01,
    )
    fig.add_trace(
         go.Waterfall(
        name = "20", orientation = "v",
        measure = measure,
        x = df_loc.index.values,
        textposition = "outside",
        #text = ["+60", "+80", "-40", "-20", "Total"],
        y = df_loc['Abrech-Ergebnis'],
        connector = {"line":{"color":"rgb(63, 63, 63)"}},
        ),
        row=2, col=1
    )
    ##################################
    df_loc = df[df[var]==loc]

    df_heat = df_loc.groupby(type_g).sum()#[['UW-Prämie','Abrech-Prämie','Ultimate-Prämie']]
    df_heat = df_heat[:len(df_heat.index.values)-2]
    fig.add_trace(
        go.Bar(
            x=df_heat.index.values,
            y=df_heat['UW-Prämie'],
            name="UW-Prämie",
            marker = {'color' : 'darkblue'}
        ),
        row=1, col=1
    )
    ###################
    fig.update_layout(
        height=300,
        width=600,
        showlegend=False,
        title_text="Prämienverlauf - "+loc,
        plot_bgcolor = "white"
    )
    return fig

@app.callback(
    dash.dependencies.Output('graph-bp', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_n=data_prep(df)
    return plots.get_bar(df_n)

@app.callback(
    dash.dependencies.Output('graph-p', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_n=data_prep(df[df['Description']==country])
    df_n['Description']=country
    return plots.get_bar(df_n)

@app.callback(
    dash.dependencies.Output('graph-ws', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    ######## Parameter
    loc = country
    var = 'Description'
    type_g = 'Renewal'
    ############

    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_loc = df[df[var]==loc]

    df_loc=df_loc.groupby(type_g).sum()
    df_loc=df_loc[:len(df_loc.index.values)-2]

    measure=[]
    for i in range(len(df_loc.index.values)-1):
        measure.append('relativ')
    measure.append('total')
    ###################################
    fig = make_subplots(
        rows=1, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.01,
    )
    fig.add_trace(
         go.Waterfall(
        name = "20", orientation = "v",
        measure = measure,
        x = df_loc.index.values,
        textposition = "outside",
        #text = ["+60", "+80", "-40", "-20", "Total"],
        y = df_loc['Abrech-Ergebnis'],
        connector = {"line":{"color":"rgb(63, 63, 63)"}},
        ),
        row=1, col=1
    )

    fig.update_layout(
        height=300,
        width=600,
        showlegend=False,
        title_text="Abwicklungsergebnis - "+loc,
        plot_bgcolor = "white"
    )
    return fig

@app.callback(
    dash.dependencies.Output('graph-wf', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    ######## Parameter
    loc = country
    type_g = 'Renewal'
    ############
    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_loc = df
    df_loc=df_loc.groupby(type_g).sum()
    df_loc=df_loc[:len(df_loc.index.values)-2]

    measure=[]
    for i in range(len(df_loc.index.values)-1):
        measure.append('relativ')
    measure.append('total')
    ###################################
    fig = make_subplots(
        rows=1, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.01,
    )
    fig.add_trace(
         go.Waterfall(
        name = "20", orientation = "v",
        measure = measure,
        x = df_loc.index.values,
        textposition = "outside",
        #text = ["+60", "+80", "-40", "-20", "Total"],
        y = df_loc['Abrech-Ergebnis'],
        connector = {"line":{"color":"rgb(63, 63, 63)"}},
        ),
        row=1, col=1
    )

    fig.update_layout(
        height=300,
        width=600,
        showlegend=False,
        title_text="Abwicklungsergebnis",
        plot_bgcolor = "white"
    )
    return fig

@app.callback(
    dash.dependencies.Output('graph-loss-p', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df_np=df[df['EboSchema']=='NP']
    df_np=df_np.groupby('Renewal').sum()[:len(df_np.index.values)-2]
    df_p=df[df['EboSchema']=='Prop']
    df_p=df_p.groupby('Renewal').sum()[:len(df_p.index.values)-2]
    #print(df_p)
    animals=df_p.index.values[:len(df_p.index.values)-2]

    loss_fig_npp = go.Figure(data=[
        go.Bar(name='NP', x=animals, y=df_np['Abrech-Schaden'], marker = {'color' : 'black'}),
        go.Bar(name='P', x=animals, y=df_p['Abrech-Schaden'], marker = {'color' : 'darkblue'})
    ])


    loss_fig_npp.update_layout(
        title_text="<b>Schadenaufteilung nach NP und P</b>", # Add figure title
        plot_bgcolor = "white", # background color
        barmode='stack' # Change the bar mode
    )
    # Set x-axis title
    loss_fig_npp.update_xaxes(title_text="<b>UW Year</b>")
    
    # legend
    loss_fig_npp.update_layout(showlegend=True)
    loss_fig_npp.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="right",
        x=1
    ))
    loss_fig_npp.update_layout(
        height=300,
        width=600,
    )
    return loss_fig_npp

@app.callback(
    dash.dependencies.Output('graph-loss', 'figure'),
    [dash.dependencies.Input('country', 'value')])
def update_figure(country):
    df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
    df=df.groupby('Renewal').sum()[:len(df.index.values)-2]
    animals=df.index.values[:len(df.index.values)-2]

    loss_fig = go.Figure(data=[
        go.Bar(x=animals, y=df['Abrech-Schaden'], marker = {'color' : 'darkblue'}),
    ])
    loss_fig.update_layout(
        title_text="<b>Schadenverlauf</b>", # Add figure title
        plot_bgcolor = "white", # background color
        #barmode='stack' # Change the bar mode
    )
    # Set x-axis title
    loss_fig.update_xaxes(title_text="<b>UW Year</b>")

    # legend
    loss_fig.update_layout(
        showlegend=False, 
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1,
            xanchor="right",
            x=1
    ))
    loss_fig.update_layout(
        height=300,
        width=600,
    )
    return loss_fig

if __name__ == '__main__':
    app.run_server(debug=False,port=8085)

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is run

 * Running on http://127.0.0.1:8085/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jun/2022 12:23:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 12:23:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 12:23:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 12:23:46] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Jun/2022 12:23:46] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Jun/2022 12:23:46] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Jun/2022 12:23:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 12:23:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 12:23:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 12:23:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 12:23:57] "POST /_dash-update-component HTTP/1.

# Bar

https://plotly.com/python/bar-charts/

In [105]:
import plotly.graph_objects as go
df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
df_np=df[df['EboSchema']=='NP']
df_np=df_np.groupby('Renewal').sum()[:len(df_np.index.values)-2]
df_p=df[df['EboSchema']=='Prop']
df_p=df_p.groupby('Renewal').sum()[:len(df_p.index.values)-2]
#print(df_p)
animals=df_p.index.values[:len(df_p.index.values)-2]

fig = go.Figure(data=[
    go.Bar(name='NP', x=animals, y=df_np['Abrech-Schaden'], marker = {'color' : 'black'}),
    go.Bar(name='P', x=animals, y=df_p['Abrech-Schaden'], marker = {'color' : 'darkblue'})
])


fig.update_layout(
    title_text="<b>Schadenaufteilung nach NP und P</b>", # Add figure title
    plot_bgcolor = "white", # background color
    barmode='stack' # Change the bar mode
)
# Set x-axis title
fig.update_xaxes(title_text="<b>UW Year</b>")

# legend
fig.update_layout(showlegend=True)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="right",
    x=1
))
fig.show()

In [106]:
df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
chart_data = pd.concat([
	df['Renewal'],
	df['Abrech-Schaden'],
	df['brch_text_kurz'],
], axis=1)

data=[]

branch = df.brch_text_kurz.unique()
for i in ['Fire','Windstorm']:
    chart_data = pd.concat([
        df['Renewal'],
        df['Abrech-Schaden'],
        df['brch_text_kurz'],
        ], axis=1)
    txt= "(`brch_text_kurz` == '"+i+"')"
    chart_data = chart_data.query(txt)
    chart_data = chart_data.sort_values(['brch_text_kurz', 'Renewal'])
    chart_data = chart_data.rename(columns={'Renewal': 'x'})
    chart_data = chart_data.rename(columns={'Abrech-Schaden': 'Abrech-Schaden'+str(i)})
    chart_data_count = chart_data.groupby(['brch_text_kurz', 'x'])[['Abrech-Schaden'+str(i)]].sum()
    x=chart_data.x.values[:len(chart_data.index.values)-2]
    y=chart_data['Abrech-Schaden'+str(i)].values[:len(chart_data.index.values)-2]
    data.append(go.Bar(name=i,x=x, y=y))
fig2 = go.Figure(data=data)
fig2.update_layout(
    title_text="<b>Schadenaufteilung nach Segment</b>", # Add figure title
    plot_bgcolor = "white", # background color
    barmode='stack' # Change the bar mode
)
#fig2.update_xaxes(title_text="<b>UW Year</b>")
fig2.update_layout(showlegend=True)
fig2.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="right",
    x=1
))
fig2.show()

# Heatmaps in Dash

In [107]:
import plotly.graph_objects as go

type_g = 'Renewal'
df_heat = df.groupby(type_g).sum()#[['UW-Prämie','Abrech-Prämie','Ultimate-Prämie']]

z=[]
for i in range(len(df_heat.index.values)):
    z.append(df_heat.iloc[i])
    
fig = go.Figure(data=go.Heatmap(
                   z=z,
                   y= df_heat.index.values,
                   x=df_heat.columns.values,
                   hoverongaps = False))
fig.show()

In [108]:
#from dash import Dash, dcc, html
import os
import pandas as pd
pd.options.display.max_columns = None
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly
import jupyter_dash as jd
import dash

#app = JupyterDash(__name__)
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.DARKLY])

df = df.reset_index().drop('index', axis=1, errors='ignore')
df.columns = [str(c) for c in df.columns]  # update columns to strings in case they are numbers

x='Renewal'
target='UW-Prämie'

chart_data = pd.concat([
	df[x],
	df[target],
], axis=1)
chart_data = chart_data.sort_values([x])
chart_data = chart_data.rename(columns={x: 'x'})
chart_data_sum = chart_data.groupby(['x'])[[target]].sum()
chart_data_sum.columns = ['sum']
chart_data = chart_data_sum.reset_index()
chart_data = chart_data.dropna()

def plot_line(x,sum):
    '''
    x:
    sum: 
    '''
    import plotly.graph_objs as go

    charts = []
    line_cfg = {'line': {'shape': 'spline', 'smoothing': 0.3}, 'mode': 'lines'}
    charts.append(go.Scatter(
        x=chart_data[x], y=chart_data[sum], name=sum, **line_cfg
    ))
    figure = go.Figure(data=charts, layout=go.Layout({
        'legend': {'orientation': 'h'},
        'title': {'text': 'Sum of UW-Prämie by Renewal'},
        'xaxis': {'tickformat': '.0f', 'title': {'text': x}},
        'yaxis': {'title': {'text': 'Sum of UW-Prämie'}, 'type': 'linear'}
    }))
    return figure

figure = plot_line('x','sum')

app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),
    html.Div([dcc.Graph(figure=figure)])

])

if __name__ == '__main__':
    app.run_server(debug=True,port=8022)

Dash is running on http://127.0.0.1:8022/

Dash is running on http://127.0.0.1:8022/

Dash is running on http://127.0.0.1:8022/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3449: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [177]:
import plotly.graph_objs as go
df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
df=df.groupby(['Description'])[['UW-Prämie']].sum()
df=df.sort_values(by='UW-Prämie',ascending=False)[:20]
charts = []
charts.append(go.Bar(
	x=df.index.values,
	y=df['UW-Prämie'],
))

figure = go.Figure(data=charts, layout=go.Layout({
    'barmode': 'group',
    'legend': {'orientation': 'h'},
    'title': {'text': 'Sum of UW-Prämie by Description'},
    'xaxis': {'nticks': 126, 'tickmode': 'auto', 'title': {'text': 'Country'}},
    'yaxis': {'title': {'text': 'Sum of UW-Prämie'}, 'type': 'linear'}
}))
figure.show()

# Pie

 https://plotly.com/python/pie-charts/

In [147]:
df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
country='Canada'
df=df[df['Description']==country]

chart_data = pd.concat([
	df['EboSchema'],
	df['UW-Prämie'],
	df['Ultimate-Prämie'],
], axis=1)
chart_data = chart_data.sort_values(['EboSchema'])
chart_data = chart_data.rename(columns={'EboSchema': 'x'})
chart_data_sum = chart_data.groupby(['x'])[['UW-Prämie', 'Ultimate-Prämie']].sum()
chart_data_sum.columns = ['UW-Prämie|sum','Ultimate-Prämie|sum']
chart_data = chart_data_sum.reset_index()
chart_data = chart_data.dropna()
chart_data = chart_data[chart_data['UW-Prämie|sum'] > 0]  # can't represent negatives in a pie

import plotly.graph_objs as go

labels=chart_data['x']
values=chart_data['UW-Prämie|sum']
chart = go.Pie(labels=labels, 
               values=values, 
               name='Sum of UW-Prämie')
figure = go.Figure(data=[chart], 
                   layout=go.Layout({'legend': {'orientation': 'h'}, 
                                     'title': {'text': 'UW-Prämie by '+country}
                                    }))
figure.show()

In [150]:
df = pd.read_pickle('data/Backtesting2020_SQ-UW_Abrech_20200907_sop.pkl')
#country='Canada'
#df=df[df['Description']==country]

chart_data = pd.concat([
	df['Renewal'],
	df['UW-Prämie'],
], axis=1)
chart_data = chart_data.sort_values(['Renewal'])
chart_data = chart_data.rename(columns={'Renewal': 'x'})
chart_data_sum = chart_data.groupby(['x'])[['UW-Prämie']].sum()
chart_data_sum.columns = ['UW-Prämie|sum']
chart_data = chart_data_sum.reset_index()
chart_data = chart_data.dropna()
chart_data = chart_data[chart_data['UW-Prämie|sum'] > 0]  # can't represent negatives in a pie

import plotly.graph_objs as go

labels=chart_data['x']
values=chart_data['UW-Prämie|sum']

chart = go.Pie(labels=labels, 
                              values=values,
                              textinfo='label + percent',
                              hole=.3,
                              pull=[0, 0, 0.2, 0],
                              name="World GDP 2007")
fig = go.Figure(data=[chart])
fig.update_layout(title_text='Prämienanteil')
fig.show()

In [161]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = ["US", "China", "European Union", "Russian Federation", "Brazil", "India",
          "Rest of World"]

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=[16, 15, 12, 6, 5, 4, 42], name="GHG Emissions"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=[27, 11, 25, 8, 1, 3, 25], name="CO2 Emissions"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")
fig.update_layout(showlegend=False)
fig.update_layout(
    title_text="Global Emissions 1990-2011",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Prämie', x=0.16, y=0.5, font_size=10, showarrow=False),
                 dict(text='Schäden', x=0.84, y=0.5, font_size=10, showarrow=False)])
fig.show()

In [152]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = ["Asia", "Europe", "Africa", "Americas", "Oceania"]

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['1980', '2007'])
fig.add_trace(go.Pie(labels=labels, values=[4, 7, 1, 7, 0.5], scalegroup='one',
                     name="World GDP 1980"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=[21, 15, 3, 19, 1], scalegroup='one',
                     name="World GDP 2007"), 1, 2)

fig.update_layout(title_text='World GDP')
fig.show()

In [174]:
import plotly.graph_objects as go

chart = go.Sunburst(
    labels=["Eve", "Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
    parents=["", "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve" ],
    values=[10, 14, 12, 10, 2, 6, 6, 4, 4],
)

fig =go.Figure(chart)
#fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig.show()